In [9]:
from dataclasses import dataclass
import pulp
import pandas as pd
import yaml

@dataclass
class Armor:
    poise: int
    weight: float
    name: str
    location: str = None
    _var = None

    @property
    def var(self):
        if self._var is None:
            self._var = pulp.LpVariable(self.name, 0, 1, pulp.LpInteger)
        return self._var


def get_armor_from_pickle(armor_type):
    df = pd.read_pickle(f"{armor_type}.pkl")
    df["Poise"] = pd.to_numeric(df["Poise"], errors='coerce')
    df["Wgt."] = pd.to_numeric(df["Wgt."], errors='coerce')
    df.fillna(0, inplace=True)
    return [Armor(name = armor["Name"], poise=armor["Poise"], weight=armor["Wgt."], location=armor["How to Acquire"] ) for armor in df.to_dict(orient='records')]


def solve(weight_limit, my_armors_only = False):
    prob = pulp.LpProblem("Poise_Optimiser", pulp.LpMaximize)

    armor_types = ["head", "chest", "arms", "legs"]
    if my_armors_only:
        with open('armor_values.yaml') as f:
            my_armors = yaml.load(f, Loader=yaml.Loader)
        typed_armor_lists = [[Armor(**armor) for armor in my_armors[armor_type]] for armor_type in armor_types]
    else:
        typed_armor_lists = [get_armor_from_pickle(armor_type) for armor_type in armor_types]

    # flatten list of lists into one big list
    armors = sum(typed_armor_lists, [])

    # limiting solution to max one of each armor type
    for armor_list in typed_armor_lists:
        prob += sum([a.var for a in armor_list]) <= 1.0

    # weight within limit
    weight = sum([armor.weight * armor.var for armor in armors])
    prob += weight <= weight_limit

    # maximise poise
    poise = sum([armor.poise * armor.var for armor in armors])
    prob += poise

    prob.solve(pulp.PULP_CBC_CMD(msg=None))

    print("Results: \n")
    print("Name - Poise - Weight ")

    for a in armors:
        if pulp.value(a.var) > 0:
            print(a.name, "-", a.poise, "-", a.weight)

    print(f"Total - {pulp.value(poise):.0f} - {pulp.value(weight):.1f}/{weight_limit:.1f}")

    print("\n", prob)
    

In [10]:
max_weight = 77.3
equiped_weight = 33.1
fat_roll_ok = False

weight_roll_threshold = 1.0 if fat_roll_ok else 0.7
free_weight = (max_weight*weight_roll_threshold)-equiped_weight

solve(weight_limit=free_weight, my_armors_only=True)

Results: 

Name - Poise - Weight 
Radahn's Redmane Helm - 9 - 5.1
Knight Armor - 19 - 10.6
Kaiden Trousers - 7 - 5.1
Total - 35 - 20.8/21.0

 Poise_Optimiser:
MAXIMIZE
2*Aristocrat_Garb + 28*Beast_Champion_Armor + 25*Beast_Champion_Armor_(Altered) + 6*Beast_Champion_Gauntlets + 17*Beast_Champion_Greaves + 9*Beast_Champion_Helm + 47*Bull_Goat_Armor + 10*Bull_Goat_Gauntlets + 28*Bull_Goat_Greaves + 15*Bull_Goat_Helm + 12*ChainArmor + 4*Chain_Coif + 3*Chain_Gauntlets + 7*Chain_Leggings + 1*Cloth_Garb + 1*Cloth_Trousers + 6*Confessor_Armor + 3*Confessor_Boots + 1*Confessor_Gloves + 2*Confessor_Hood + 14*Exile_Armor + 3*Exile_Gauntlets + 8*Exile_Greaves + 4*Exile_Hood + 4*Godrick_Soldier_Gauntlets + 3*Kaiden_Gauntlets + 7*Kaiden_Trousers + 19*Knight_Armor + 4*Knight_Gauntlets + 11*Knight_Greaves + 6*Knight_Helm + 11*Leyndell_Knight_Greaves + 1*Noble's_Gloves + 2*Noble's_Traveling_Gear + 1*Noble's_Trousers + 6*Radahn's_Gauntlets + 17*Radahn's_Greaves + 28*Radahn's_Lion_Armor + 9*Radahn's_Red